In [93]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostClassifier, cv

Преобразовали данные

In [79]:
df1 = pd.read_csv('21-22.csv', sep=';')
def create_df(df):
    df = df.iloc[:-4]
    df = df[['Wk', 'Day', 'Date', 'Time', 'Home', 'Score', 'Away', 'Referee']]
    df['Wk'] = df['Wk'].astype(int)
    df[['Home_score', 'Away_score']] = df['Score'].str.split('–', expand=True)
    df[['Home_score', 'Away_score']] = df[['Home_score', 'Away_score']].astype(int) 
    df['Month'] = df['Date'].apply(lambda x: x[3:5])
    df['Hours'] = df['Time'].apply(lambda x: x[:2]).astype(int)
    df = df.drop(['Score', 'Date', 'Time'], axis=1)
    df['target'] = np.where(df['Home_score'] > df['Away_score'], 1, 
                            np.where(df['Home_score'] < df['Away_score'], -1, 0))
    # 1 - победа хозяев, 0 - ничья, -1 - победа гостей
    return df
df1 = create_df(df1)
df1

,Wk,Day,Home,Away,Referee,Home_score,Away_score,Month,Hours,target
0,1,Fri,Rostov,Dynamo Mosc,Vladimir Moskalev,0,2,07,20,-1
1,1,Sat,FC Khimki,Zenit,Evgeny Kukulyak,1,3,07,17,-1
2,1,Sat,Rubin Kazan,Spartak Moscow,Vladislav Bezborodov,1,0,07,20,1
3,1,Sat,Loko Moscow,Arsenal Tula,Igor Panin,3,1,07,20,1
4,1,Sun,Ural,Krasnodar,Vasiliy Kazartsev,0,3,07,19,-1
...,...,...,...,...,...,...,...,...,...,...
235,30,Sat,FC Khimki,Spartak Moscow,Sergey Karasev,2,1,05,17,1
236,30,Sat,CSKA Moscow,Rostov,Yan Bobrovskiy,4,0,05,17,1
237,30,Sat,Dynamo Mosc,Sochi,Vladimir Moskalev,1,5,05,17,-1
238,30,Sat,Krasnodar,Akhmat Grozny,Pavel Shadykhanov,1,1,05,17,0


In [80]:
Total_20_21 = {'Rostov': 9, 'FC Khimki': 8, 'Rubin Kazan': 4, 'Loko Moscow': 3, 'Ural': 12,
       'Samara': None, 'CSKA Moscow': 6, 'Nizhny Novgorod': None, 'Arsenal Tula': 14, 'Ufa':13,
       'Krasnodar':10, 'Akhmat Grozny':11, 'Zenit':1, 'Spartak Moscow':2,
       'Dynamo Mosc':7, 'Sochi':5}
df1['Total_last_year_Home'] = df1['Home'].apply(lambda x: Total_20_21[x])
df1['Total_last_year_Away'] = df1['Away'].apply(lambda x: Total_20_21[x])
df1

,Wk,Day,Home,Away,Referee,Home_score,Away_score,Month,Hours,target,Total_last_year_Home,Total_last_year_Away
0,1,Fri,Rostov,Dynamo Mosc,Vladimir Moskalev,0,2,07,20,-1,9.0,7.0
1,1,Sat,FC Khimki,Zenit,Evgeny Kukulyak,1,3,07,17,-1,8.0,1.0
2,1,Sat,Rubin Kazan,Spartak Moscow,Vladislav Bezborodov,1,0,07,20,1,4.0,2.0
3,1,Sat,Loko Moscow,Arsenal Tula,Igor Panin,3,1,07,20,1,3.0,14.0
4,1,Sun,Ural,Krasnodar,Vasiliy Kazartsev,0,3,07,19,-1,12.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
235,30,Sat,FC Khimki,Spartak Moscow,Sergey Karasev,2,1,05,17,1,8.0,2.0
236,30,Sat,CSKA Moscow,Rostov,Yan Bobrovskiy,4,0,05,17,1,6.0,9.0
237,30,Sat,Dynamo Mosc,Sochi,Vladimir Moskalev,1,5,05,17,-1,7.0,5.0
238,30,Sat,Krasnodar,Akhmat Grozny,Pavel Shadykhanov,1,1,05,17,0,10.0,11.0


In [87]:
df2 = pd.read_csv('22-23.csv', sep=';')
df2 = create_df(df2)
Total_21_22 = {'FC Khimki':13, 'CSKA Moscow':5, 'Orenburg':None, 'Akhmat Grozny':7,
       'Torpedo Moscow':None, 'Loko Moscow':6, 'Dynamo Mosc':3, 'Krasnodar':4,
       'Zenit':1, 'Ural':12, 'Fakel Voronezh':None, 'Sochi':2, 'Nizhny Novgorod':11,
       'Spartak Moscow':10, 'Samara':8, 'Rostov':9}
df2['Total_last_year_Home'] = df2['Home'].apply(lambda x: Total_21_22[x])
df2['Total_last_year_Away'] = df2['Away'].apply(lambda x: Total_21_22[x])
df2

,Wk,Day,Home,Away,Referee,Home_score,Away_score,Month,Hours,target,Total_last_year_Home,Total_last_year_Away
0,1,Fri,FC Khimki,Zenit,Evgeny Kukulyak,1,1,07,20,0,13.0,1.0
1,1,Sat,CSKA Moscow,Ural,Vitaly Meshkov,2,0,07,15,1,5.0,12.0
2,1,Sat,Orenburg,Samara,Pavel Shadykhanov,2,4,07,19,-1,NaN,8.0
3,1,Sat,Akhmat Grozny,Spartak Moscow,Vladislav Bezborodov,1,1,07,20,0,7.0,10.0
4,1,Sun,Torpedo Moscow,Sochi,Sergey Ivanov,1,3,07,15,-1,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
235,30,Sat,Akhmat Grozny,Krasnodar,Rafael Shafeev,2,2,06,17,0,7.0,4.0
236,30,Sat,CSKA Moscow,Rostov,Pavel Kukuyan,4,1,06,17,1,5.0,9.0
237,30,Sat,Samara,Spartak Moscow,Aleksey Amelin,1,0,06,18,1,8.0,10.0
238,30,Sat,Orenburg,Dynamo Mosc,Sergey Ivanov,3,0,06,19,1,NaN,3.0


In [88]:
df = pd.concat([df1, df2], axis=0)
df

,Wk,Day,Home,Away,Referee,Home_score,Away_score,Month,Hours,target,Total_last_year_Home,Total_last_year_Away
0,1,Fri,Rostov,Dynamo Mosc,Vladimir Moskalev,0,2,07,20,-1,9.0,7.0
1,1,Sat,FC Khimki,Zenit,Evgeny Kukulyak,1,3,07,17,-1,8.0,1.0
2,1,Sat,Rubin Kazan,Spartak Moscow,Vladislav Bezborodov,1,0,07,20,1,4.0,2.0
3,1,Sat,Loko Moscow,Arsenal Tula,Igor Panin,3,1,07,20,1,3.0,14.0
4,1,Sun,Ural,Krasnodar,Vasiliy Kazartsev,0,3,07,19,-1,12.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
235,30,Sat,Akhmat Grozny,Krasnodar,Rafael Shafeev,2,2,06,17,0,7.0,4.0
236,30,Sat,CSKA Moscow,Rostov,Pavel Kukuyan,4,1,06,17,1,5.0,9.0
237,30,Sat,Samara,Spartak Moscow,Aleksey Amelin,1,0,06,18,1,8.0,10.0
238,30,Sat,Orenburg,Dynamo Mosc,Sergey Ivanov,3,0,06,19,1,NaN,3.0


In [90]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338 entries, 0 to 239
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Wk                    338 non-null    int64  
 1   Day                   338 non-null    object 
 2   Home                  338 non-null    object 
 3   Away                  338 non-null    object 
 4   Referee               338 non-null    object 
 5   Home_score            338 non-null    int64  
 6   Away_score            338 non-null    int64  
 7   Month                 338 non-null    object 
 8   Hours                 338 non-null    int64  
 9   target                338 non-null    int64  
 10  Total_last_year_Home  338 non-null    float64
 11  Total_last_year_Away  338 non-null    float64
dtypes: float64(2), int64(5), object(5)
memory usage: 34.3+ KB


In [91]:
df

,Wk,Day,Home,Away,Referee,Home_score,Away_score,Month,Hours,target,Total_last_year_Home,Total_last_year_Away
0,1,Fri,Rostov,Dynamo Mosc,Vladimir Moskalev,0,2,07,20,-1,9.0,7.0
1,1,Sat,FC Khimki,Zenit,Evgeny Kukulyak,1,3,07,17,-1,8.0,1.0
2,1,Sat,Rubin Kazan,Spartak Moscow,Vladislav Bezborodov,1,0,07,20,1,4.0,2.0
3,1,Sat,Loko Moscow,Arsenal Tula,Igor Panin,3,1,07,20,1,3.0,14.0
4,1,Sun,Ural,Krasnodar,Vasiliy Kazartsev,0,3,07,19,-1,12.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
234,30,Sat,Nizhny Novgorod,FC Khimki,Vasiliy Kazartsev,2,0,06,17,1,11.0,13.0
235,30,Sat,Akhmat Grozny,Krasnodar,Rafael Shafeev,2,2,06,17,0,7.0,4.0
236,30,Sat,CSKA Moscow,Rostov,Pavel Kukuyan,4,1,06,17,1,5.0,9.0
237,30,Sat,Samara,Spartak Moscow,Aleksey Amelin,1,0,06,18,1,8.0,10.0
